### LLM 및 MCP Client 초기화

In [44]:
from chatterer import BaseMessage, Chatterer, HumanMessage, AIMessage, SystemMessage  # noqa
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

from rich import print

# Chatterer.from_provider를 사용하여 LLM Client를 초기화한다.
llm = Chatterer.from_provider(
    "openai:gpt-5-nano",
    option={
        "kwargs": {
            "reasoning_effort": "minimal",
        },
    },
)

# Langchain과 호환되는 MCP Client를 초기화한다. 이때, 여기에 정의된 MCP Server가 이미 실행되어 있어야 한다.
mcp_client = MultiServerMCPClient({
    "test-mcp-server": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http",
    }
})

### MCP Client에서, MCP Server에 있는 툴들을 읽어들인다

In [45]:
# 이 툴들은 MCP Server에서 정의된 툴들이다
tools = await mcp_client.get_tools()

# 읽어들인 툴들을 출력한다
for tool_call in tools:
    print(tool_call)

StructuredTool(
    name='add',
    description='Adds two integer numbers together.',
    args_schema={
        'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
        'required': ['a', 'b'],
        'type': 'object'
    },
    metadata={'_meta': {'_fastmcp': {'tags': []}}},
    response_format='content_and_artifact',
    coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001D408A21E40>
)

StructuredTool(
    name='get_weather',
    description='Get the weather of the city.',
    args_schema={'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'object'},
    metadata={'_meta': {'_fastmcp': {'tags': []}}},
    response_format='content_and_artifact',
    coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001D408A22C00>
)

StructuredTool(
    name='search_products',
    description='Search the product catalog.',
    args_schema={
        'properties': {
            'query': {'type': 'string'},
            'max_results': {'default': 10, 'type': 'integer'},
            'sort_by': {'default': 'relevance', 'type': 'string'},
            'category': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None}
        },
        'required': ['query'],
        'type': 'object'
    },
    metadata={'_meta': {'_fastmcp': {'tags': []}}},
    response_format='content_and_artifact',
    coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001D408A23E20>
)

### MCP Client가 읽어들인 툴을 사용하는 LLM Agent를 생성한다

In [46]:
agent = create_agent(llm.client, tools)

### LLM Agent가 해당 툴들을 적재적소에 잘 사용하는지 확인한다.

In [47]:
agent_invoke_result = await agent.ainvoke({
    "messages": [
        HumanMessage(content="현재 서울 날씨가 어떤가?"),
    ]
})


for message_no, message in enumerate(agent_invoke_result["messages"]):
    print(message)

HumanMessage(
    content='현재 서울 날씨가 어떤가?',
    additional_kwargs={},
    response_metadata={},
    id='9dfa9c38-7bf1-4cfb-bb36-2ab9343f39e4'
)

AIMessage(
    content='',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 24,
            'prompt_tokens': 210,
            'total_tokens': 234,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_provider': 'openai',
        'model_name': 'gpt-5-nano-2025-08-07',
        'system_fingerprint': None,
        'id': 'chatcmpl-Co14Cv4eqd5s9jbY8QjpGPJ0Hbolv',
        'service_tier': 'default',
        'finish_reason': 'tool_calls',
        'logprobs': None
    },
    id='lc_run--019b2ffb-037d-7800-ab3c-971ff699730f-0',
    tool_calls=[
        {
            'name': 'get_weather',
            'args': {'city': 'Seoul'},
            'id': 'call_MhaE2WU8dOrtKLQEXVBa9KNd',
            'type': 'tool_call'
        }
    ],
    usage_metadata={
        'input_tokens': 210,
        'output_tokens': 24,
        'total_tokens': 234,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

ToolMessage(
    content=[
        {
            'type': 'text',
            'text': 'The weather of Seoul is sunny and 26℃',
            'id': 'lc_d080ec66-f586-4576-a4c8-b213583a9391'
        }
    ],
    name='get_weather',
    id='8d2243f2-f485-4857-ae25-aaa97ca073be',
    tool_call_id='call_MhaE2WU8dOrtKLQEXVBa9KNd',
    artifact={'structured_content': {'result': 'The weather of Seoul is sunny and 26℃'}}
)

AIMessage(
    content='서울의 현재 날씨: 맑고 기온은 26°C 입니다. \n\n다른 지역의 날씨나 더 자세한 정보(오늘의 습도, 바람 
방향/세기 등)가 필요하시면 알려주세요.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 59,
            'prompt_tokens': 249,
            'total_tokens': 308,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_provider': 'openai',
        'model_name': 'gpt-5-nano-2025-08-07',
        'system_fingerprint': None,
        'id': 'chatcmpl-Co14F9bf3TTalJZEqYidDcsTvV0ju',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='lc_run--019b2ffb-0d0d-78c3-bdec-37bb021ce20b-0',
    usage_metadata={
        'input_tokens': 249,
        'output_tokens': 59,
        'total_tokens': 308,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)